In [1]:
# -------------------- Standard Library Imports --------------------
import argparse
import glob
import os
import sys
from multiprocessing import Pool, cpu_count
from random import sample
from subprocess import PIPE, Popen, STDOUT, call, run

# -------------------- Scientific Libraries ------------------------
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import pybedtools
import pysam
import scipy
import statsmodels.stats.multitest as smm
from Bio import SeqIO

# -------------------- Visualization Libraries ---------------------
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patches as mpatches
import matplotlib.ticker as mtick
from matplotlib.offsetbox import AnchoredText
import matplotlib.colors as clr
from matplotlib import cm
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
from matplotlib_venn import venn2, venn2_circles

# -------------------- Matplotlib Configuration --------------------
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica Neue'

# -------------------- PyBedTools Temp Directory -------------------
pybedtools.helpers.set_tempdir('/fs/cbsuhy02/storage/jz855/tmp/') 


In [2]:
# Set the project root directory (assuming current working directory is within a subdirectory)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))


# OL13_ENCSR394HXI

In [ ]:
# -------------------------------------------------------------
# Load and process barcode count data from Tiling MPRA (OL13_ENCSR394HXI)
# -------------------------------------------------------------

# Define project path and input file
data_path = os.path.join(
    project_root, 'data', 'raw_data', 'TilingMPRA', 
    'OL13_ENCSR394HXI', 'ENCFF048CMC.tsv'
)

# Load raw barcode-level count data
count = pd.read_csv(data_path, sep='\t')

# Count how many barcodes each oligo has
barcode_counts = count.groupby('Oligo').size()

# Filter oligos with fewer than 10 barcodes
oligos_lt_10 = barcode_counts[barcode_counts < 10].index.tolist()

# Print and optionally save
print(f"Number of oligos with <10 barcodes: {len(oligos_lt_10)}")

# Save to file if needed
out_path = os.path.join(
    project_root, 'data', 'count_data', 'TilingMPRA', 
    'OL13_ENCSR394HXI', 'oligos_with_lt10_barcodes.txt'
)
with open(out_path, 'w') as f:
    for oligo in oligos_lt_10:
        f.write(f"{oligo}\n")

# Group by Oligo sequence and sum counts across all barcodes (vectorized, efficient)
count = (
    count
    .groupby('Oligo')[['plasmid_r1', 'plasmid_r2', 'plasmid_r3', 'plasmid_r4',
                       'K562_r1', 'K562_r2', 'K562_r3', 'K562_r4']]
    .sum()
    .rename(columns={
        'plasmid_r1': 'DNA1', 'plasmid_r2': 'DNA2', 'plasmid_r3': 'DNA3', 'plasmid_r4': 'DNA4',
        'K562_r1': 'RNA1', 'K562_r2': 'RNA2', 'K562_r3': 'RNA3', 'K562_r4': 'RNA4'
    })
    .reset_index()
)

print(f"Unique elements after aggregation: {len(count)}")

# -------------------------------------------------------------
# Map oligos to hg38 coordinates using processed BED file
# -------------------------------------------------------------
bed_path = os.path.join(
    project_root, 'data', 'raw_data', 'TilingMPRA', 
    'OL13_ENCSR394HXI', 'OL13_hg38.bed'
)
ol13_hg38 = pd.read_csv(bed_path, sep='\t', header=None)
ol13_hg38 = ol13_hg38.set_index(3)[[0, 1, 2]]  # Use oligo ID as index

# Join aggregated counts with genomic coordinates
count = count.set_index('Oligo')
merged = count.join(ol13_hg38, how='inner')  # Use inner join to drop unmatched
merged.columns.name = None
print(f"After joining with hg38: {len(merged)}")

# -------------------------------------------------------------
# Load attribute file with strand and project labels
# -------------------------------------------------------------
attr_path = os.path.join(
    project_root, 'data', 'raw_data', 'TilingMPRA', 
    'OL13_ENCSR394HXI', 'OL13_ENCSR394HXI.attributes'
)
attribute = pd.read_csv(attr_path, sep='\t').set_index('ID')

print("Project types:\n", attribute['project'].value_counts())

# Merge attributes (strand, project)
merged = merged.join(attribute[['strand', 'project']], how='inner')
merged = merged.reset_index()  # Bring 'Oligo' back as a column

# -------------------------------------------------------------
# Finalize and format DataFrame
# -------------------------------------------------------------
merged.columns = ['Oligo', 'DNA1', 'DNA2', 'DNA3', 'DNA4', 'RNA1', 'RNA2', 'RNA3', 'RNA4',
                  'seqnames', 'start', 'end', 'strand', 'project']

# Annotate control elements
merged['annot'] = merged['project'].apply(lambda x: 'neg_ctrl' if x == 'negCtrl' else '-')

# Format strand and coordinate columns
merged['strand'] = merged['strand'].map({'rev': '-', 'fwd': '+'})
merged['start'] = merged['start'].astype('Int64')
merged['end'] = merged['end'].astype('Int64')

# Keep only relevant columns and desired project types
final_data = merged[['seqnames', 'start', 'end', 'Oligo', 'strand', 'project', 'annot',
                     'DNA1', 'DNA2', 'DNA3', 'DNA4', 'RNA1', 'RNA2', 'RNA3', 'RNA4']]
final_data = final_data[final_data['project'].isin(['Tiles', 'negCtrl'])]

print("Annotation counts:\n", final_data['annot'].value_counts())
print(f"Final data length: {len(final_data)}")

# -------------------------------------------------------------
# Save final processed data
# -------------------------------------------------------------
out_path = os.path.join(
    project_root, 'data', 'count_data', 'TilingMPRA', 
    'OL13_ENCSR394HXI', 'processed_OL13_count_with_all_annotation.txt'
)
final_data.to_csv(out_path, sep='\t', index=False)
print('finished')


Number of oligos with <10 barcodes: 1131
Unique elements after aggregation: 55233
After joining with hg38: 54011
Project types:
 project
Tiles        44330
SNV          12432
negCtrl        150
emVarCtrl       80
expCtrl         18
Name: count, dtype: int64
Annotation counts:
 annot
-           42714
neg_ctrl      150
Name: count, dtype: int64
Final data length: 42864
finished


# OL43_ENCSR917SFD

In [ ]:
# -------------------------------------------------------------
# Load and process barcode count data from Tiling MPRA (OL43_ENCSR917SFD)
# -------------------------------------------------------------

# Define project path and input file
data_path = os.path.join(
    project_root, 'data', 'raw_data', 'TilingMPRA', 
    'OL43_ENCSR917SFD', 'ENCFF267VJA.tsv'
)

# Load raw barcode-level count data
count = pd.read_csv(data_path, sep='\t')

# Count how many barcodes each oligo has
barcode_counts = count.groupby('Oligo').size()

# Filter oligos with fewer than 10 barcodes
oligos_lt_10 = barcode_counts[barcode_counts < 10].index.tolist()

# Print and optionally save
print(f"Number of oligos with <10 barcodes: {len(oligos_lt_10)}")

# Save to file if needed
out_path = os.path.join(
    project_root, 'data', 'count_data', 'TilingMPRA', 
    'OL43_ENCSR917SFD', 'oligos_with_lt10_barcodes.txt'
)
with open(out_path, 'w') as f:
    for oligo in oligos_lt_10:
        f.write(f"{oligo}\n")

# Group by Oligo sequence and sum counts across all barcodes (vectorized, efficient)
count = (
    count
    .groupby('Oligo')[['plasmid_r1', 'plasmid_r2', 'plasmid_r3', 'plasmid_r4', 'plasmid_r5', 
                       'K562_r1', 'K562_r2', 'K562_r3', 'K562_r4', 'K562_r5']]
    .sum()
    .rename(columns={
        'plasmid_r1': 'DNA1', 'plasmid_r2': 'DNA2', 'plasmid_r3': 'DNA3', 'plasmid_r4': 'DNA4', 'plasmid_r5': 'DNA5', 
        'K562_r1': 'RNA1', 'K562_r2': 'RNA2', 'K562_r3': 'RNA3', 'K562_r4': 'RNA4', 'K562_r5': 'RNA5'
    })
    .reset_index()
)

print(f"Unique oligos after aggregation: {len(count)}")

count = count.set_index('Oligo')

# -------------------------------------------------------------
# Load attribute file with strand and project labels
# -------------------------------------------------------------
attr_path = os.path.join(
    project_root, 'data', 'raw_data', 'TilingMPRA', 
    'OL43_ENCSR917SFD', 'OL43_ENCSR917SFD.attributes'
)
attribute = pd.read_csv(attr_path, sep='\t').set_index('ID')

print("Project types:\n", attribute['project'].value_counts())

# Merge attributes (strand, project)
merged = count.join(attribute[['chr', 'start', 'stop', 'strand', 'project']], how='left')
merged = merged.reset_index()  # Bring 'Oligo' back as a column

# -------------------------------------------------------------
# Finalize and format DataFrame
# -------------------------------------------------------------
merged.columns = ['Oligo', 'DNA1', 'DNA2', 'DNA3', 'DNA4', 'DNA5', 'RNA1', 'RNA2', 'RNA3', 'RNA4', 'RNA5', 
                  'seqnames', 'start', 'end', 'strand', 'project']

# Annotate control elements
merged['annot'] = merged['project'].apply(lambda x: 'neg_ctrl' if x == 'ctrl_neg-Yu-ORF' else '-')

# Format strand and coordinate columns
merged['strand'] = merged['strand'].map({'rev': '-', 'fwd': '+'})
merged['seqnames'] = merged['seqnames'].fillna('.')
merged['seqnames'] = ['chr'+str(x) if x != '.' else '.' for x in merged['seqnames']]
merged['start'] = merged['start'].fillna(-1)
merged['end'] = merged['end'].fillna(-1)
merged['start'] = merged['start'].astype('Int64')
merged['end'] = merged['end'].astype('Int64')

# Keep only relevant columns and desired project types
final_data = merged[['seqnames', 'start', 'end', 'Oligo', 'strand', 'project', 'annot',
                     'DNA1', 'DNA2', 'DNA3', 'DNA4', 'DNA5', 'RNA1', 'RNA2', 'RNA3', 'RNA4', 'RNA5']]
# final_data = final_data[final_data['project'].isin(['Tiles', 'negCtrl'])]

print("Annotation counts:\n", final_data['annot'].value_counts())
print(f"Final data length: {len(final_data)}")

# -------------------------------------------------------------
# Save final processed data
# -------------------------------------------------------------
out_path = os.path.join(
    project_root, 'data', 'count_data', 'TilingMPRA', 
    'OL43_ENCSR917SFD', 'processed_OL43_count_with_all_annotation.txt'
)
final_data.to_csv(out_path, sep='\t', index=False)
print('finished')



Number of oligos with <10 barcodes: 2485
Unique oligos after aggregation: 99306
Project types:
 project
Tiles-GATA                     54520
Tiles-MYC                      42090
ctrl_neg-Yu-ORF                 1876
ctrl_neg                         506
ctrl_neg-Shendure-shuffle        250
ctrl_pos-Shendure-Gasperini      145
ctrl_neg-Shendure-Gasperini      138
ctrl_neg-Shendure-Ernst          100
ctrl_emvar                        77
ctrl_pos                          72
ctrl_pos-Shendure-Ernst           50
ctrl_pos-Shendure-Manually        19
ctrl_neg-Shendure-MYC             11
ctrl_emvar,ctrl_pos               10
ctrl_pos,ctrl_emvar                9
ctrl_neg-Shendure-GATA             2
Name: count, dtype: int64


<ipython-input-9-d6e4ca81db33>:57: DtypeWarning: Columns (1,2,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  attribute = pd.read_csv(attr_path, sep='\t').set_index('ID')


Annotation counts:
 annot
-           97436
neg_ctrl     1870
Name: count, dtype: int64
Final data length: 99306
finished


# OL45_ENCSR363XER

In [17]:
# -------------------------------------------------------------
# Load and process barcode count data from Tiling MPRA (OL43_ENCSR917SFD)
# -------------------------------------------------------------

# Define project path and input file
data_path = os.path.join(
    project_root, 'data', 'raw_data', 'TilingMPRA', 
    'OL45_ENCSR363XER', 'ENCFF130WRM.tsv'
)

# Load raw barcode-level count data
count = pd.read_csv(data_path, sep='\t')

# Count how many barcodes each oligo has
barcode_counts = count.groupby('Oligo').size()

# Filter oligos with fewer than 10 barcodes
oligos_lt_10 = barcode_counts[barcode_counts < 10].index.tolist()

# Print and optionally save
print(f"Number of oligos with <10 barcodes: {len(oligos_lt_10)}")

# Save to file if needed
out_path = os.path.join(
    project_root, 'data', 'count_data', 'TilingMPRA', 
    'OL45_ENCSR363XER', 'oligos_with_lt10_barcodes.txt'
)
with open(out_path, 'w') as f:
    for oligo in oligos_lt_10:
        f.write(f"{oligo}\n")

# Group by Oligo sequence and sum counts across all barcodes (vectorized, efficient)
count = (
    count
    .groupby('Oligo')[['plasmid_r1', 'plasmid_r2', 'plasmid_r3', 'plasmid_r4', 'plasmid_r5', 
                       'K562_r1', 'K562_r2', 'K562_r3', 'K562_r4']]
    .sum()
    .rename(columns={
        'plasmid_r1': 'DNA1', 'plasmid_r2': 'DNA2', 'plasmid_r3': 'DNA3', 'plasmid_r4': 'DNA4', 'plasmid_r5': 'DNA5', 
        'K562_r1': 'RNA1', 'K562_r2': 'RNA2', 'K562_r3': 'RNA3', 'K562_r4': 'RNA4'
    })
    .reset_index()
)

print(f"Unique oligos after aggregation: {len(count)}")

count = count.set_index('Oligo')

# -------------------------------------------------------------
# Load attribute file with strand and project labels
# -------------------------------------------------------------
attr_path = os.path.join(
    project_root, 'data', 'raw_data', 'TilingMPRA', 
    'OL45_ENCSR363XER', 'ENCFF179FXK.tsv'
)
attribute = pd.read_csv(attr_path, sep='\t').set_index('ID')

print("Project types:\n", attribute['project'].value_counts())

# Merge attributes (strand, project)
merged = count.join(attribute[['chr', 'start', 'stop', 'strand', 'project']], how='left')
merged = merged.reset_index()  # Bring 'Oligo' back as a column

# -------------------------------------------------------------
# Finalize and format DataFrame
# -------------------------------------------------------------
merged.columns = ['Oligo', 'DNA1', 'DNA2', 'DNA3', 'DNA4', 'DNA5', 'RNA1', 'RNA2', 'RNA3', 'RNA4', 
                  'seqnames', 'start', 'end', 'strand', 'project']

# Annotate control elements
merged['annot'] = merged['project'].apply(lambda x: 'neg_ctrl' if x == 'ctrl_neg-Yu-ORF' else '-')

# Format strand and coordinate columns
merged['strand'] = merged['strand'].map({'rev': '-', 'fwd': '+'})
merged['seqnames'] = merged['seqnames'].fillna('.')
merged['seqnames'] = ['chr'+str(x) if x != '.' else '.' for x in merged['seqnames']]
merged['start'] = merged['start'].fillna(-1)
merged['end'] = merged['end'].fillna(-1)
merged['start'] = merged['start'].astype('Int64')
merged['end'] = merged['end'].astype('Int64')

# Keep only relevant columns and desired project types
final_data = merged[['seqnames', 'start', 'end', 'Oligo', 'strand', 'project', 'annot',
                     'DNA1', 'DNA2', 'DNA3', 'DNA4', 'DNA5', 'RNA1', 'RNA2', 'RNA3', 'RNA4']]

print("Annotation counts:\n", final_data['annot'].value_counts())
print(f"Final data length: {len(final_data)}")

# -------------------------------------------------------------
# Save final processed data
# -------------------------------------------------------------
out_path = os.path.join(
    project_root, 'data', 'count_data', 'TilingMPRA', 
    'OL45_ENCSR363XER', 'processed_OL45_count_with_all_annotation.txt'
)
final_data.to_csv(out_path, sep='\t', index=False)
print('finished')



Number of oligos with <10 barcodes: 2189
Unique oligos after aggregation: 94381


<ipython-input-17-0c8af819b307>:56: DtypeWarning: Columns (1,2,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  attribute = pd.read_csv(attr_path, sep='\t').set_index('ID')


Project types:
 project
Tiles-LMO2                                                     20003
Tiles-RBM38                                                    20003
Tiles-BCL11A                                                   20003
Tiles-HBE1                                                     20003
Tiles-HBA2                                                     11630
ctrl_neg-Yu-ORF                                                 1876
ctrl_neg                                                         506
ctrl_neg-Shendure-shuffle                                        250
ctrl_pos-Shendure-Gasperini                                      145
ctrl_neg-Shendure-Gasperini                                      138
ctrl_neg-Shendure-Ernst                                          100
ctrl_emvar                                                        70
ctrl_pos                                                          69
ctrl_pos-Shendure-Ernst                                           50
ctrl_pos-S